# CPSS IDENTIFICATION NOTEBOOK
Integrates enhanced detection for:
- EACS (Electronic Access Control Systems) + BAS
- VSS (Video Surveillance Systems)
- I&HAS (Intrusion & Hold-up Alarm Systems)

Features:
- 40 brands total (100% thesis Appendix C coverage)
- Protocol detection (RTSP, ONVIF, BACnet, SIA DC-09, etc.)
- HTTP path matching (70+ paths)
- Model number detection (100+ patterns)
- Confidence scoring (0-100%)
- Multi-function device handling
- BAS subcategory flagging
- Enhanced cloud/IT exclusions

**Brand Coverage:**
- **VSS**: 15 brands (Hikvision, Dahua, Axis, MOBOTIX, Geutebruck, etc.)
- **EACS**: 13 brands (Nedap, Paxton, Genetec, + 4 BAS brands)
- **I&HAS**: 12 brands (AJAX, Vanderbilt, Honeywell, Bosch, etc.)
- **Total**: 40 brands with 100% thesis Appendix C coverage

**Device Categories:**
1. **EACS** - Electronic Access Control Systems
   - Includes BAS (Building Automation Systems) as subcategory
   - Multi-function flag for devices like Genetec
2. **VSS** - Video Surveillance Systems
   - IP cameras, NVRs, DVRs, VMS platforms
3. **I&HAS** - Intrusion & Hold-up Alarm Systems
   - Alarm panels, intrusion detection, sensors

**Expected Runtime:** ~15 minutes
**Expected Precision:** 85-95%

## Configuration and Setup

### Import & Setup

In [14]:
# ========================================
# CELL 2: IMPORTS & SETUP
# ========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import sys
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 10

# Output directory
output_dir = Path('./output/2_cpss_identification')
output_dir.mkdir(exist_ok=True)

# Color palette
COLORS = {
    'eacs': '#7C3AED',      # Purple
    'vss': '#14B8A6',       # Turquoise
    'ihas': '#F59E0B',      # Yellow
    'quaternary': '#22C55E', # Emerald
    'danger': '#EF4444',
    'neutral': '#9CA3AF',
}

print(f"Enhanced CPSS Identification started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Output directory: {output_dir.absolute()}")

Enhanced CPSS Identification started: 2026-01-01 15:06:47
Output directory: H:\_HHS_thesis\GitHub\thesis\v1\output\2_cpss_identification


### Load Data

In [15]:
# ========================================
# CELL 3: LOAD DATA
# ========================================

print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

df = pd.read_csv('./staging/3_prepare_analyses/modat_service_all.csv', low_memory=False)
print(f"Loaded {len(df):,} total services")
print(f"Dataset has {len(df.columns)} columns")
print("="*70)



LOADING DATA
Loaded 271,255 total services
Dataset has 175 columns


### Load Enhanced Detection Modules

In [16]:
# ========================================
# CELL 4: LOAD ENHANCED DETECTION MODULES (BETTER VERSION)
# ========================================
# Dit lost de "unresolved reference" warnings op

import importlib.util
import sys

print("\n" + "="*70)
print("LOADING ENHANCED DETECTION MODULES")
print("="*70)

# Function to load modules properly
def load_detection_module(module_name, file_path):
    """Load a Python module and return it"""
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None:
        raise ImportError(f"Could not load {module_name} from {file_path}")
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

# Load the three detection modules
print("\nLoading EACS enhanced detection...")
eacs_mod = load_detection_module('eacs_enhanced', '6__EACS_enhanced_detection.py')

print("\nLoading I&HAS enhanced detection...")
ihas_mod = load_detection_module('ihas_enhanced', '6__IHAS_enhanced_detection.py')

print("\nLoading VSS enhanced detection...")
vss_mod = load_detection_module('vss_enhanced', '6__VSS_enhanced_detection.py')

# Extract the functions and configs (NOW IDE WILL RECOGNIZE THEM)
identify_eacs_enhanced = eacs_mod.identify_eacs_enhanced
EACS_ENHANCED_CONFIG = eacs_mod.EACS_ENHANCED_CONFIG

identify_ihas_enhanced = ihas_mod.identify_ihas_enhanced
IHAS_ENHANCED_CONFIG = ihas_mod.IHAS_ENHANCED_CONFIG

identify_vss_enhanced = vss_mod.identify_vss_enhanced
VSS_ENHANCED_CONFIG = vss_mod.VSS_ENHANCED_CONFIG

print("\nAll enhanced detection modules loaded")
print(f"  EACS: {len(EACS_ENHANCED_CONFIG['brands'])} brands, {len(EACS_ENHANCED_CONFIG['protocols'])} protocols")
print(f"  I&HAS: {len(IHAS_ENHANCED_CONFIG['brands'])} brands, {len(IHAS_ENHANCED_CONFIG['protocols'])} protocols")
print(f"  VSS: {len(VSS_ENHANCED_CONFIG['brands'])} brands, {len(VSS_ENHANCED_CONFIG['protocols'])} protocols")
print("="*70)


LOADING ENHANCED DETECTION MODULES

Loading EACS enhanced detection...
Enhanced EACS detection loaded (STRICT VERSION)
  Brands: 13
  HTTP paths: 35+ (specific only)
  Protocols: 3 (with banner confirmation)

  CHANGES FROM PREVIOUS:
  - NO single-word matches (edge, evo, space)
  - Require brand + product combination
  - Enhanced exclusions for false positives
  - Port detection requires banner confirmation

  Expected: Lower recall, MUCH higher precision
  Goal: Zero false positives

Loading I&HAS enhanced detection...
Enhanced I&HAS detection loaded (STRICT VERSION)
  Brands: 10
  HTTP paths: 25+ (product-specific)
  Protocols: 2 (with banner confirmation)

  CHANGES:
  - NO generic terms (alarm, security, sensor)
  - Require brand + specific product
  - Detailed match reporting in output
  - Enhanced exclusions

  Expected: Much lower count, high precision

Loading VSS enhanced detection...
Enhanced VSS detection loaded (STRICT VERSION)
  Brands: 11
  HTTP paths: 35+ (camera/NVR s

## Run Identification

In [18]:
# ========================================
# CELL 5: RUN ENHANCED IDENTIFICATION
# ========================================

print("\n" + "="*70)
print("RUNNING ENHANCED CPSS IDENTIFICATION")
print("="*70)

print("\n""1. Identifying EACS devices...")
eacs_results = df.apply(lambda row: identify_eacs_enhanced(row), axis=1)
df['is_eacs'] = eacs_results.apply(lambda x: x[0])
df['eacs_confidence'] = eacs_results.apply(lambda x: x[1])
df['eacs_reason'] = eacs_results.apply(lambda x: x[2])

# Add BAS subcategory flag
df['is_bas'] = df['eacs_reason'].str.contains('bacnet|lon|modbus|desigo|metasys|struxureware|enteliweb', case=False, na=False)

print("EACS identification complete")

print("\n2. Identifying VSS devices...")
vss_results = df.apply(lambda row: identify_vss_enhanced(row), axis=1)
df['is_vss'] = vss_results.apply(lambda x: x[0])
df['vss_confidence'] = vss_results.apply(lambda x: x[1])
df['vss_reason'] = vss_results.apply(lambda x: x[2])

print("VSS identification complete")

print("\n3. Identifying IHAS devices...")
ihas_results = df.apply(lambda row: identify_ihas_enhanced(row), axis=1)
df['is_ihas'] = ihas_results.apply(lambda x: x[0])
df['ihas_confidence'] = ihas_results.apply(lambda x: x[1])
df['ihas_reason'] = ihas_results.apply(lambda x: x[2])

print("IHAS identification complete")

# Extract datasets
eacs_df = df[df['is_eacs']].copy()
vss_df = df[df['is_vss']].copy()
ihas_df = df[df['is_ihas']].copy()

# Flag multi-function devices
df['is_multi_function'] = (df['is_eacs'].astype(int) + df['is_vss'].astype(int) + df['is_ihas'].astype(int)) > 1

multi_function_df = df[df['is_multi_function']].copy()

print("\n" + "="*70)
print("IDENTIFICATION RESULTS")
print("="*70)
print(f"\nTotal services in dataset: {len(df):,}")
print(f"\nEACS identified:   {len(eacs_df):>8,} ({len(eacs_df)/len(df)*100:>5.2f}%)")
print(f"  └─ BAS devices:  {eacs_df['is_bas'].sum():>8,} ({eacs_df['is_bas'].sum()/len(eacs_df)*100 if len(eacs_df)>0 else 0:>5.2f}% of EACS)")
print(f"VSS identified:    {len(vss_df):>8,} ({len(vss_df)/len(df)*100:>5.2f}%)")
print(f"IHAS identified:  {len(ihas_df):>8,} ({len(ihas_df)/len(df)*100:>5.2f}%)")

total_cpss = len(df[(df['is_eacs']) | (df['is_vss']) | (df['is_ihas'])])
print(f"\nTotal CPSS:        {total_cpss:>8,} ({total_cpss/len(df)*100:>5.2f}%)")
print(f"Multi-function:    {len(multi_function_df):>8,}")

if len(multi_function_df) > 0:
    print(f"\nMulti-function breakdown:")
    overlap_eacs_vss = len(df[df['is_eacs'] & df['is_vss']])
    overlap_eacs_ihas = len(df[df['is_eacs'] & df['is_ihas']])
    overlap_vss_ihas = len(df[df['is_vss'] & df['is_ihas']])

    if overlap_eacs_vss > 0:
        print(f"  EACS + VSS:     {overlap_eacs_vss:>8,}")
    if overlap_eacs_ihas > 0:
        print(f"  EACS + IHAS:   {overlap_eacs_ihas:>8,}")
    if overlap_vss_ihas > 0:
        print(f"  VSS + IHAS:    {overlap_vss_ihas:>8,}")

print("="*70)


RUNNING ENHANCED CPSS IDENTIFICATION

1. Identifying EACS devices...


KeyError: 0

### Confidence Analysis

In [ ]:
# ========================================
# CELL 6: CONFIDENCE ANALYSIS
# ========================================

print("\n" + "="*70)
print("CONFIDENCE SCORE ANALYSIS")
print("="*70)

def analyze_confidence(df_subset, device_type, confidence_col):
    """Analyze confidence score distribution"""
    if len(df_subset) == 0:
        return

    print(f"\n{device_type} Confidence Scores:")
    print("-" * 60)

    avg_conf = df_subset[confidence_col].mean()
    min_conf = df_subset[confidence_col].min()
    max_conf = df_subset[confidence_col].max()

    print(f"  Average:  {avg_conf:.1f}%")
    print(f"  Range:    {min_conf:.0f}% - {max_conf:.0f}%")

    # Confidence bands
    very_high = len(df_subset[df_subset[confidence_col] >= 90])
    high = len(df_subset[(df_subset[confidence_col] >= 80) & (df_subset[confidence_col] < 90)])
    medium = len(df_subset[(df_subset[confidence_col] >= 70) & (df_subset[confidence_col] < 80)])
    low = len(df_subset[df_subset[confidence_col] < 70])

    total = len(df_subset)

    print(f"\n  Confidence bands:")
    print(f"    Very High (≥90%):  {very_high:>6,} ({very_high/total*100:>5.1f}%)")
    print(f"    High (80-89%):     {high:>6,} ({high/total*100:>5.1f}%)")
    print(f"    Medium (70-79%):   {medium:>6,} ({medium/total*100:>5.1f}%)")
    print(f"    Low (<70%):        {low:>6,} ({low/total*100:>5.1f}%)")

analyze_confidence(eacs_df, 'EACS', 'eacs_confidence')
analyze_confidence(vss_df, 'VSS', 'vss_confidence')
analyze_confidence(ihas_df, 'IHAS', 'ihas_confidence')

print("="*70)

### Detection Method

In [ ]:
# ========================================
# CELL 7: DETECTION METHOD BREAKDOWN
# ========================================

print("\n" + "="*70)
print("DETECTION METHOD ANALYSIS")
print("="*70)

def analyze_detection_methods(df_subset, device_type, reason_col):
    """Analyze how devices were detected"""
    if len(df_subset) == 0:
        return

    print(f"\n{device_type} Detection Methods:")
    print("-" * 60)

    # Categorize detection methods
    protocol_count = len(df_subset[df_subset[reason_col].str.contains('protocol:', na=False)])
    tag_count = len(df_subset[df_subset[reason_col].str.contains('tag:', na=False)])
    http_path_count = len(df_subset[df_subset[reason_col].str.contains('http_path:', na=False)])
    brand_count = len(df_subset[df_subset[reason_col].str.contains('brand:', na=False)])
    product_count = len(df_subset[df_subset[reason_col].str.contains('product:', na=False)])
    model_count = len(df_subset[df_subset[reason_col].str.contains('model:', na=False)])
    cert_count = len(df_subset[df_subset[reason_col].str.contains('cert:', na=False)])
    keyword_count = len(df_subset[df_subset[reason_col].str.contains('keyword', na=False)])

    total = len(df_subset)

    print(f"  Protocol detection:    {protocol_count:>6,} ({protocol_count/total*100:>5.1f}%)")
    print(f"  Tag match:             {tag_count:>6,} ({tag_count/total*100:>5.1f}%)")
    print(f"  HTTP path match:       {http_path_count:>6,} ({http_path_count/total*100:>5.1f}%)")
    print(f"  Brand match:           {brand_count:>6,} ({brand_count/total*100:>5.1f}%)")
    print(f"  Product match:         {product_count:>6,} ({product_count/total*100:>5.1f}%)")
    print(f"  Model number match:    {model_count:>6,} ({model_count/total*100:>5.1f}%)")
    print(f"  Certificate match:     {cert_count:>6,} ({cert_count/total*100:>5.1f}%)")
    print(f"  Keyword match:         {keyword_count:>6,} ({keyword_count/total*100:>5.1f}%)")

    # Top specific reasons
    print(f"\n  Top 10 specific detection reasons:")
    reasons = df_subset[reason_col].value_counts().head(10)
    for reason, count in reasons.items():
        if not str(reason).startswith('excluded'):
            print(f"    {str(reason)[:55]:<55} {count:>6,}")

analyze_detection_methods(eacs_df, 'EACS', 'eacs_reason')
analyze_detection_methods(vss_df, 'VSS', 'vss_reason')
analyze_detection_methods(ihas_df, 'IHAS', 'ihas_reason')

print("="*70)

## Brand Distribution

In [ ]:
# ========================================
# CELL 8: BRAND DISTRIBUTION
# ========================================

print("\n" + "="*70)
print("BRAND DISTRIBUTION ANALYSIS")
print("="*70)

def extract_brand_from_reason(reason):
    """Extract brand name from match reason"""
    if pd.isna(reason):
        return 'Unknown'

    reason_str = str(reason)

    # Extract from brand:, product:, cert:, http_path:, model:
    for prefix in ['brand:', 'product:', 'cert:', 'http_path:', 'model:']:
        if prefix in reason_str:
            start = reason_str.index(prefix) + len(prefix)
            end = reason_str.find(':', start)
            if end == -1:
                end = reason_str.find('(', start)
            if end == -1:
                end = len(reason_str)
            brand = reason_str[start:end].strip()
            return brand

    if 'protocol:' in reason_str:
        return 'Protocol Match'
    if 'tag:' in reason_str:
        return 'Tag Match'
    if 'keyword' in reason_str:
        return 'Keyword Match'

    return 'Unknown'

def analyze_brand_distribution(df_subset, device_type, reason_col):
    """Analyze brand distribution"""
    if len(df_subset) == 0:
        return df_subset

    df_subset['detected_brand'] = df_subset[reason_col].apply(extract_brand_from_reason)
    brand_counts = df_subset['detected_brand'].value_counts()

    print(f"\n{device_type} Brand Distribution:")
    print("-" * 60)
    print(f"{'Brand':<30} {'Count':>10} {'%':>8}")
    print("-" * 60)

    for brand, count in brand_counts.head(15).items():
        pct = (count / len(df_subset) * 100)
        print(f"{brand:<30} {count:>10,} {pct:>7.2f}%")

    if len(brand_counts) > 15:
        other = brand_counts.iloc[15:].sum()
        print(f"{'Others':<30} {other:>10,} {other/len(df_subset)*100:>7.2f}%")

    return df_subset

eacs_df = analyze_brand_distribution(eacs_df, 'EACS', 'eacs_reason')
vss_df = analyze_brand_distribution(vss_df, 'VSS', 'vss_reason')
ihas_df = analyze_brand_distribution(ihas_df, 'IHAS', 'ihas_reason')

print("="*70)

## CPSS outliers analyses

### BAS Subcategory analysis

In [ ]:
# ========================================
# CELL 11: BAS SUBCATEGORY ANALYSIS
# ========================================

if eacs_df['is_bas'].sum() > 0:
    print("\n" + "="*70)
    print("BAS SUBCATEGORY ANALYSIS")
    print("="*70)

    bas_df = eacs_df[eacs_df['is_bas']].copy()

    print(f"\nBAS devices: {len(bas_df):,} ({len(bas_df)/len(eacs_df)*100:.1f}% of EACS)")
    print(f"Pure EACS: {len(eacs_df) - len(bas_df):,} ({(len(eacs_df) - len(bas_df))/len(eacs_df)*100:.1f}% of EACS)")

    # BAS brands
    if 'detected_brand' in bas_df.columns:
        print("\nBAS brands:")
        bas_brands = bas_df['detected_brand'].value_counts()
        for brand, count in bas_brands.items():
            print(f"  {brand:<20} {count:>6,}")

    # BAS protocols
    bas_protocols = bas_df['eacs_reason'].str.extract(r'protocol:(\w+)', expand=False).value_counts()
    if len(bas_protocols) > 0:
        print("\nBAS protocols:")
        for protocol, count in bas_protocols.items():
            if pd.notna(protocol):
                print(f"  {protocol:<20} {count:>6,}")

    print("="*70)

### Multi-function Device Analysis

In [ ]:
# ========================================
# CELL 12: MULTI-FUNCTION DEVICE ANALYSIS
# ========================================

if len(multi_function_df) > 0:
    print("\n" + "="*70)
    print("MULTI-FUNCTION DEVICE ANALYSIS")
    print("="*70)

    print(f"\nMulti-function devices: {len(multi_function_df):,}")

    # Category combinations
    print("\nCategory combinations:")
    for idx, row in multi_function_df.head(10).iterrows():
        ip = row.get('ip', 'N/A')
        categories = []
        if row.get('is_eacs', False):
            categories.append(f"EACS({row.get('eacs_confidence', 0):.0f}%)")
        if row.get('is_vss', False):
            categories.append(f"VSS({row.get('vss_confidence', 0):.0f}%)")
        if row.get('is_ihas', False):
            categories.append(f"IHAS({row.get('ihas_confidence', 0):.0f}%)")

        brand = row.get('detected_brand', 'Unknown')
        print(f"  {ip:<20} {brand:<20} {' + '.join(categories)}")

    if len(multi_function_df) > 10:
        print(f"  ... and {len(multi_function_df) - 10} more")

    print("="*70)

## CSV Exports

In [ ]:
# ========================================
# CELL 9: CSV EXPORTS
# ========================================

print("\n" + "="*70)
print("EXPORTING CSV FILES")
print("="*70)

# Define export columns (if they exist)
export_columns_base = [
    'ip', 'service.port', 'service.protocol',
    'service.http.title', 'service.fingerprints.product', 'service.fingerprints.vendor',
    'service.banner', 'asn.org', 'geo.country', 'geo.city',
    'service.fingerprints.tags'
]

def export_cpss_category(df_subset, device_type, confidence_col, reason_col):
    """Export CPSS category to CSV with confidence scores"""
    if len(df_subset) == 0:
        print(f"\n⚠️  No {device_type} devices to export")
        return

    # Find available columns
    export_cols = [col for col in export_columns_base if col in df_subset.columns]

    # Add CPSS-specific columns
    export_cols.extend([confidence_col, reason_col])

    if 'detected_brand' in df_subset.columns:
        export_cols.append('detected_brand')

    # Add BAS flag for EACS
    if device_type == 'EACS' and 'is_bas' in df_subset.columns:
        export_cols.append('is_bas')

    # Add multi-function flag
    if 'is_multi_function' in df_subset.columns:
        export_cols.append('is_multi_function')

    # Export
    filename = f'cpss_{device_type.lower()}_services_enhanced.csv'
    filepath = output_dir / filename
    df_subset[export_cols].to_csv(filepath, index=False)

    print(f"\n {device_type} exported: {filename}")
    print(f"  Services: {len(df_subset):,}")
    print(f"  Avg confidence: {df_subset[confidence_col].mean():.1f}%")
    print(f"  Columns: {len(export_cols)}")

    # High confidence sample
    high_conf = df_subset[df_subset[confidence_col] >= 90]
    if len(high_conf) > 0:
        print(f"\n  High confidence sample (≥90%, first 3):")
        for idx, row in high_conf.head(3).iterrows():
            ip = row.get('ip', 'N/A')
            port = row.get('service.port', 'N/A')
            brand = row.get('detected_brand', 'N/A')
            conf = row.get(confidence_col, 0)
            reason = str(row.get(reason_col, ''))[:40]
            print(f"    {ip}:{port} | {brand:<15} | {conf:>3.0f}% | {reason}")

# Export each category
export_cpss_category(eacs_df, 'EACS', 'eacs_confidence', 'eacs_reason')
export_cpss_category(vss_df, 'VSS', 'vss_confidence', 'vss_reason')
export_cpss_category(ihas_df, 'IHAS', 'ihas_confidence', 'ihas_reason')

# Export multi-function devices
if len(multi_function_df) > 0:
    multi_export_cols = [col for col in export_columns_base if col in multi_function_df.columns]
    multi_export_cols.extend(['is_eacs', 'is_vss', 'is_ihas',
                              'eacs_confidence', 'vss_confidence', 'ihas_confidence',
                              'eacs_reason', 'vss_reason', 'ihas_reason'])

    filepath = output_dir / 'cpss_multi_function_devices.csv'
    multi_function_df[multi_export_cols].to_csv(filepath, index=False)
    print(f"\n Multi-function devices exported: cpss_multi_function_devices.csv")
    print(f"  Services: {len(multi_function_df):,}")

# Export combined CPSS dataset
all_cpss = pd.concat([
    eacs_df.assign(cpss_primary_category='EACS'),
    vss_df.assign(cpss_primary_category='VSS'),
    ihas_df.assign(cpss_primary_category='IHAS')
], ignore_index=True)

if len(all_cpss) > 0:
    export_cols = [col for col in export_columns_base if col in all_cpss.columns]
    export_cols.extend(['cpss_primary_category',
                       'eacs_confidence', 'vss_confidence', 'ihas_confidence',
                       'eacs_reason', 'vss_reason', 'ihas_reason'])

    if 'detected_brand' in all_cpss.columns:
        export_cols.append('detected_brand')
    if 'is_bas' in all_cpss.columns:
        export_cols.append('is_bas')
    if 'is_multi_function' in all_cpss.columns:
        export_cols.append('is_multi_function')

    filepath = output_dir / 'cpss_all_services_enhanced.csv'
    all_cpss[[col for col in export_cols if col in all_cpss.columns]].to_csv(filepath, index=False)
    print(f"\n Combined CPSS exported: cpss_all_services_enhanced.csv")
    print(f"  Total services: {len(all_cpss):,}")

print("="*70)

## Visualisations

In [ ]:
# ========================================
# CELL 10: VISUALIZATIONS
# ========================================

print("\n" + "="*70)
print("CREATING VISUALIZATIONS")
print("="*70)

fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. CPSS Category Distribution (Pie)
ax1 = fig.add_subplot(gs[0, 0])
cpss_counts = pd.Series({
    'EACS': len(eacs_df),
    'VSS': len(vss_df),
    'IHAS': len(ihas_df)
})
colors_pie = [COLORS['eacs'], COLORS['vss'], COLORS['ihas']]
wedges, texts, autotexts = ax1.pie(cpss_counts.values, labels=cpss_counts.index,
                                     autopct='%1.1f%%', colors=colors_pie, startangle=90)
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)
ax1.set_title('CPSS Category Distribution', fontsize=14, fontweight='bold', pad=15)

# 2. Services by Category (Bar)
ax2 = fig.add_subplot(gs[0, 1])
bars = ax2.bar(cpss_counts.index, cpss_counts.values, color=colors_pie,
               edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Number of Services', fontsize=11)
ax2.set_title('CPSS Services by Category', fontsize=14, fontweight='bold', pad=15)
ax2.grid(axis='y', alpha=0.3)
for bar, count in zip(bars, cpss_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, count,
            f'{count:,}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 3. Confidence Score Distribution
ax3 = fig.add_subplot(gs[0, 2])
confidence_data = []
confidence_labels = []
if len(eacs_df) > 0:
    confidence_data.append(eacs_df['eacs_confidence'].values)
    confidence_labels.append('EACS')
if len(vss_df) > 0:
    confidence_data.append(vss_df['vss_confidence'].values)
    confidence_labels.append('VSS')
if len(ihas_df) > 0:
    confidence_data.append(ihas_df['ihas_confidence'].values)
    confidence_labels.append('IHAS')

if confidence_data:
    bp = ax3.boxplot(confidence_data, labels=confidence_labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors_pie[:len(confidence_data)]):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    ax3.set_ylabel('Confidence Score (%)', fontsize=11)
    ax3.set_title('Confidence Score Distribution', fontsize=14, fontweight='bold', pad=15)
    ax3.grid(axis='y', alpha=0.3)
    ax3.set_ylim(0, 105)

# 4. Detection Method Comparison
ax4 = fig.add_subplot(gs[1, :])
detection_methods = {
    'Protocol': [],
    'HTTP Path': [],
    'Brand/Product': [],
    'Tag': [],
    'Keyword': []
}

for df_subset, reason_col, label in [(eacs_df, 'eacs_reason', 'EACS'),
                                      (vss_df, 'vss_reason', 'VSS'),
                                      (ihas_df, 'ihas_reason', 'IHAS')]:
    if len(df_subset) > 0:
        total = len(df_subset)
        detection_methods['Protocol'].append(len(df_subset[df_subset[reason_col].str.contains('protocol:', na=False)]) / total * 100)
        detection_methods['HTTP Path'].append(len(df_subset[df_subset[reason_col].str.contains('http_path:', na=False)]) / total * 100)
        detection_methods['Brand/Product'].append(len(df_subset[df_subset[reason_col].str.contains('brand:|product:|model:', na=False)]) / total * 100)
        detection_methods['Tag'].append(len(df_subset[df_subset[reason_col].str.contains('tag:', na=False)]) / total * 100)
        detection_methods['Keyword'].append(len(df_subset[df_subset[reason_col].str.contains('keyword', na=False)]) / total * 100)
    else:
        for key in detection_methods:
            detection_methods[key].append(0)

x = np.arange(len(['EACS', 'VSS', 'IHAS']))
width = 0.15
multiplier = 0

for method, values in detection_methods.items():
    offset = width * multiplier
    bars = ax4.bar(x + offset, values, width, label=method)
    multiplier += 1

ax4.set_ylabel('Percentage (%)', fontsize=11)
ax4.set_title('Detection Method Comparison by Category', fontsize=14, fontweight='bold', pad=15)
ax4.set_xticks(x + width * 2)
ax4.set_xticklabels(['EACS', 'VSS', 'IHAS'])
ax4.legend(loc='upper right', fontsize=10)
ax4.grid(axis='y', alpha=0.3)

# 5. Top Brands (Combined)
ax5 = fig.add_subplot(gs[2, :])
all_brands = []
if len(eacs_df) > 0 and 'detected_brand' in eacs_df.columns:
    all_brands.extend(eacs_df['detected_brand'].tolist())
if len(vss_df) > 0 and 'detected_brand' in vss_df.columns:
    all_brands.extend(vss_df['detected_brand'].tolist())
if len(ihas_df) > 0 and 'detected_brand' in ihas_df.columns:
    all_brands.extend(ihas_df['detected_brand'].tolist())

if all_brands:
    brand_counts = pd.Series(all_brands).value_counts().head(15)
    ax5.barh(range(len(brand_counts)), brand_counts.values, color=COLORS['quaternary'])
    ax5.set_yticks(range(len(brand_counts)))
    ax5.set_yticklabels(brand_counts.index)
    ax5.set_xlabel('Number of Services', fontsize=11)
    ax5.set_title('Top 15 CPSS Brands (All Categories)', fontsize=14, fontweight='bold', pad=15)
    ax5.grid(axis='x', alpha=0.3)

    for i, (brand, count) in enumerate(brand_counts.items()):
        ax5.text(count, i, f' {count:,}', va='center', fontsize=9)

plt.suptitle('Enhanced CPSS Identification Summary', fontsize=18, fontweight='bold', y=0.995)
plt.savefig(output_dir / 'cpss_identification_enhanced_summary.jpg', dpi=300, bbox_inches='tight')
plt.show()

print("\n Visualization saved: cpss_identification_enhanced_summary.jpg")
print("="*70)

## Summary Report

In [ ]:
# ========================================
# CELL 13: SUMMARY REPORT
# ========================================

print("\n" + "="*70)
print("GENERATING SUMMARY REPORT")
print("="*70)

summary_report = f"""
ENHANCED CPSS IDENTIFICATION SUMMARY REPORT
{'='*70}
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET OVERVIEW
{'-'*70}
Total services analyzed:          {len(df):>12,}

CPSS IDENTIFICATION RESULTS
{'-'*70}
EACS (Access Control):            {len(eacs_df):>12,} ({len(eacs_df)/len(df)*100:>6.2f}%)
  └─ BAS subcategory:             {eacs_df['is_bas'].sum():>12,} ({eacs_df['is_bas'].sum()/len(eacs_df)*100 if len(eacs_df)>0 else 0:>6.2f}% of EACS)
  └─ Average confidence:          {eacs_df['eacs_confidence'].mean() if len(eacs_df)>0 else 0:>12.1f}%

VSS (Video Surveillance):         {len(vss_df):>12,} ({len(vss_df)/len(df)*100:>6.2f}%)
  └─ Average confidence:          {vss_df['vss_confidence'].mean() if len(vss_df)>0 else 0:>12.1f}%

IHAS (Intrusion & Alarm):        {len(ihas_df):>12,} ({len(ihas_df)/len(df)*100:>6.2f}%)
  └─ Average confidence:          {ihas_df['ihas_confidence'].mean() if len(ihas_df)>0 else 0:>12.1f}%

Total CPSS devices:               {total_cpss:>12,} ({total_cpss/len(df)*100:>6.2f}%)
Multi-function devices:           {len(multi_function_df):>12,}
Non-CPSS services:                {len(df)-total_cpss:>12,} ({(len(df)-total_cpss)/len(df)*100:>6.2f}%)

ENHANCEMENT FEATURES APPLIED
{'-'*70}
 Protocol detection (RTSP, ONVIF, BACnet, SIA DC-09, Contact ID)
 HTTP path pattern matching
 Model number detection
 Confidence scoring system (0-100%)
 Multi-function device handling
 BAS subcategory flagging
 Enhanced cloud provider exclusions

TOP DETECTION METHODS
{'-'*70}
EACS:  {eacs_df['eacs_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).index[0] if len(eacs_df)>0 else 'N/A':<30} {eacs_df['eacs_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).values[0] if len(eacs_df)>0 else 0:>6,}
VSS:   {vss_df['vss_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).index[0] if len(vss_df)>0 else 'N/A':<30} {vss_df['vss_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).values[0] if len(vss_df)>0 else 0:>6,}
IHAS: {ihas_df['ihas_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).index[0] if len(ihas_df)>0 else 'N/A':<30} {ihas_df['ihas_reason'].str.extract(r'^(\w+):', expand=False).value_counts().head(1).values[0] if len(ihas_df)>0 else 0:>6,}

FILES GENERATED
{'-'*70}
cpss_eacs_services_enhanced.csv   {len(eacs_df):>12,} services
cpss_vss_services_enhanced.csv    {len(vss_df):>12,} services
cpss_ihas_services_enhanced.csv   {len(ihas_df):>12,} services
cpss_all_services_enhanced.csv    {total_cpss:>12,} services
cpss_multi_function_devices.csv   {len(multi_function_df):>12,} services
cpss_identification_enhanced_summary.jpg

NEXT STEPS
{'-'*70}
1. Validate CSV exports
2. Review high-confidence matches (≥90%)
3. Investigate multi-function devices
4. Proceed to ISO controls assessment (analyses_3)

{'='*70}
End of Report
"""

print(summary_report)

# Save report
report_path = output_dir / 'cpss_identification_enhanced_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(summary_report)

print(f"\n Summary report saved: cpss_identification_enhanced_report.txt")
print("="*70)

print("\n" + "="*70)
print("ENHANCED CPSS IDENTIFICATION COMPLETE")
print("="*70)
print(f"\n All files saved to: {output_dir.absolute()}")
print("\nEnhancement Summary:")
print(f"  • {len(eacs_df):,} EACS devices (avg {eacs_df['eacs_confidence'].mean():.1f}% confidence)")
print(f"  • {len(vss_df):,} VSS devices (avg {vss_df['vss_confidence'].mean():.1f}% confidence)")
print(f"  • {len(ihas_df):,} IHAS devices (avg {ihas_df['ihas_confidence'].mean():.1f}% confidence)")
print(f"  • {total_cpss:,} total CPSS devices identified")
print("\nReview the CSV files and proceed to ISO control assessment!")
print("="*70)